In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.cluster import DBSCAN
import similaritymeasures
import pickle
from pathlib import Path
from shapely.ops import LineString
from tqdm import tqdm

import gps_utils

In [2]:
import json
config = json.load((Path.cwd() / 'config.json').open('rb'))
export_fp = Path(config['project_directory']) / 'CycleAtlanta'
if export_fp.exists() == False:
    export_fp.mkdir()
    
project_dir = Path(config['project_directory']) / "CycleAtlanta"
user_data_definitions = json.load(open(Path.cwd()/'user_data_definition.json'))
cycleatlanta_data_filepath = Path(config['cycleatlanta'])

## Reduce Spacing for Map Matching

In [3]:
with (project_dir/'coords_1.pkl').open('rb') as fh:
    coords_dict = pickle.load(fh)

In [4]:
for tripid, coords in coords_dict.items():
    reduced = gps_utils.reduce_spacing(coords,speed_mph_thresh=2,spacing_ft=50)
    #overwrite
    coords_dict[tripid] = reduced
    # speed_mph_thresh=2
    # spacing_ft=50

    # if coords['speed_mph'].isna().any() == False:
    #     coords = coords[coords['speed_mph'].abs()>speed_mph_thresh]
    
    # coords = gps_utils.calculate_coordinate_metrics(coords)
    
    # current_spacing = spacing_ft
    
    # #start with first point
    # keep = [coords.index.tolist()[0]]
    
    # for index, value in coords['traversed_distance_ft'].items():
    #     if value > current_spacing:
    #         keep.append(index)
    #         current_spacing = value + spacing_ft
    
    # #remove points
    # coords = coords.loc[keep]


In [5]:
with (project_dir/'reduced_spacing.pkl').open('wb') as fh:
    pickle.dump(coords_dict,fh)

## Reduce Resolution for Frechet Distance and Visualization

In [6]:
with (project_dir/'coords_1.pkl').open('rb') as fh:
    coords_dict = pickle.load(fh)

In [7]:
for tripid, coords in coords_dict.items():
    line = np.array(LineString(coords_dict[tripid]['geometry'].tolist()).simplify(100).coords)
    coords_dict[tripid] = line

#pre-computation
#trip_lines = {tripid: np.array(LineString(coords_dict[tripid]['geometry'].tolist()).simplify(100).coords) for tripid in need_geo}b

In [8]:
with (project_dir/'rdp.pkl').open('wb') as fh:
    pickle.dump(coords_dict,fh)